In [2]:
#!pip install nest_asyncio 

In [3]:
#!pip install httpx

In [4]:
#!pip install tqdm

In [5]:
import requests
import pandas as pd
import re
from tqdm import tqdm

API_KEY = "ddee2cca4c****c5ab8ae6332093b164" # (masked) spoonacular api key
URL = "https://api.spoonacular.com/recipes/analyze"

def extract_selected_nutrients(analysis_response):
    nutrients = analysis_response.get("nutrition", {}).get("nutrients", [])
    
    selected = {
        "Saturated Fat": None,
        "Sodium": None,
        "Sugar": None
    }
    
    for nutrient in nutrients:
        name = nutrient.get("name", "").lower()
        amount = nutrient.get("amount")
        unit = nutrient.get("unit")
        
        if "saturated fat" in name:
            selected["Saturated Fat"] = f"{amount} {unit}"
        elif "sodium" in name:
            selected["Sodium"] = f"{amount} {unit}"
        elif "sugar" in name:
            selected["Sugar"] = f"{amount} {unit}"
    
    return selected

def analyze_recipe(recipe_data, include_nutrition=True, language="en"):
    headers = {
        "Content-Type": "application/json"
    }
    params = {
        "apiKey": API_KEY,
        "includeNutrition": str(include_nutrition).lower()
    }
    body = {
        "title": recipe_data.get("title", ""),
        "servings": recipe_data.get("servings", 1),
        "ingredients": recipe_data.get("ingredients", []),
        "language": language,
    }
    
    response = requests.post(URL, headers=headers, params=params, json=body)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def main():
    df = pd.read_excel("output_with_ingredients.xlsx")

    if "ingredients" not in df.columns or "title" not in df.columns or "video_id" not in df.columns:
        print("Error: Required columns ('ingredients', 'title', 'video_id') not found in the Excel file.")
        return

    try:
        start_index = int(input("Enter the starting row index (0-based): ").strip())
        if start_index < 0 or start_index >= len(df):
            raise ValueError("Starting index is out of range.")
    except Exception as e:
        print(f"Invalid input: {e}")
        return

    results = []
    end_index = min(start_index + 2800, len(df)) # 2000 rows max
    output_file = f"recipe_nutrition_output_{start_index}_to_{end_index-1}.xlsx"

    try:
        for index in tqdm(range(start_index, end_index), desc="Processing rows"):
            row = df.iloc[index]
            raw_ingredients = row["ingredients"]
            recipe_title = row["title"]
            video_id = row["video_id"]

            # Extract serving size
            try:
                serving_size = int(row.get("serving size", 1))
                if serving_size <= 0:
                    raise ValueError
            except:
                serving_size = 1
            
            entry = {
                "Video ID": video_id,
                "Title": recipe_title,
                "Saturated Fat": None,
                "Sodium": None,
                "Sugar": None,
                "Status": "Failed"
            }

            try:
                if not isinstance(raw_ingredients, str) or raw_ingredients.strip() == "":
                    raise ValueError("Ingredients is not a valid non-empty string.")

                ingredients_list = [
                    item.strip() for item in re.split(r'\s{2,}|\n+', raw_ingredients)
                    if item.strip()
                ]
                if not ingredients_list:
                    raise ValueError("No ingredients found after splitting.")
            except Exception as e:
                print(f"Row {index}: Failed to parse ingredients list: {e}")
                results.append(entry)
                continue

            recipe = {
                "title": recipe_title,
                "servings": serving_size,
                "ingredients": ingredients_list
            }

            analysis = analyze_recipe(recipe, include_nutrition=True)
            if analysis is None:
                print(f"Row {index}: Failed to analyze recipe.")
                results.append(entry)
                continue

            selected_nutrients = extract_selected_nutrients(analysis)
            entry.update(selected_nutrients)
            entry["Status"] = "Success"
            results.append(entry)

    except KeyboardInterrupt:
        print("\n\n Process interrupted by user. Saving partial results...")

    finally:
        if results:
            columns_order = ["Video ID", "Title", "Saturated Fat", "Sodium", "Sugar", "Status"]
            nutrient_df = pd.DataFrame(results)[columns_order]
            nutrient_df.to_excel(output_file, index=False)
            print(f"\n Results saved to {output_file} (processed rows {start_index} to {start_index + len(results) - 1})")
        else:
            print("No data was processed or saved.")

if __name__ == "__main__":
    main()


Enter the starting row index (0-based):  0


Processing rows:   0%|                                                                        | 0/2724 [00:00<?, ?it/s]

Row 0: Failed to parse ingredients list: Ingredients is not a valid non-empty string.


Processing rows:   0%|                                                                | 3/2724 [00:02<43:37,  1.04it/s]

Row 3: Failed to parse ingredients list: Ingredients is not a valid non-empty string.


...

Row 2715: Failed to parse ingredients list: Ingredients is not a valid non-empty string.
Row 2716: Failed to parse ingredients list: Ingredients is not a valid non-empty string.


Processing rows: 100%|████████████████████████████████████████████████████████████▊| 2718/2724 [37:02<00:02,  2.64it/s]

Row 2718: Failed to parse ingredients list: Ingredients is not a valid non-empty string.


Processing rows: 100%|█████████████████████████████████████████████████████████████| 2724/2724 [37:06<00:00,  1.22it/s]


Row 2723: Failed to parse ingredients list: Ingredients is not a valid non-empty string.

✅ Results saved to recipe_nutrition_output_0_to_2723.xlsx (processed rows 0 to 2723)
